<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Functions" data-toc-modified-id="Functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Car-Pole-Example" data-toc-modified-id="Car-Pole-Example-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Car Pole Example</a></span><ul class="toc-item"><li><span><a href="#Pure-Randomness" data-toc-modified-id="Pure-Randomness-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Pure Randomness</a></span></li><li><span><a href="#Pole-Velocity-Error" data-toc-modified-id="Pole-Velocity-Error-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Pole Velocity Error</a></span></li></ul></li></ul></div>

# Car Pole Example

The main idea of this notebook is to interact with the `car_pole` from [Open AI Gym](https://gym.openai.com/) and treat different algorithms for this problem.

In [12]:
#Loading the required libreries
import gym
import numpy as np
import xgboost
import itertools
import math
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score, StratifiedKFold
from gym import wrappers

import io
import base64
from IPython.display import HTML

## Functions 

This section has all the function that I will use in the notebook.

In [2]:
#Genera un expand_grid para hacer validacion cruzada
def expand_grid(*itrs):
   product = list(itertools.product(*itrs))
   return pd.DataFrame({'Var{}'.format(i+1):[x[i] for x in product] for i in range(len(itrs))})

## Car Pole Example

This section covers the `car_pole` example.

In [18]:
#My envorinement
game = 'Car Pole'
my_env = gym.make('CartPole-v0')
my_env = wrappers.Monitor(my_env, '../Gym Videos/' + game, force=True)

In [19]:
#Reset the game, the car in this case
my_env.reset()

DependencyNotInstalled: Found neither the ffmpeg nor avconv executables. On OS X, you can install ffmpeg via `brew install ffmpeg`. On most Ubuntu variants, `sudo apt-get install ffmpeg` should do it. On Ubuntu 14.04, however, you'll need to install avconv with `sudo apt-get install libav-tools`.

### Pure Randomness

Here we do 1000 iterations and average the result just picking an action between 0 and 1 randomly.

In [10]:
#Exercise: Determine how many steps, on average, are taken, when
#actions are randomly sampled
steps_array = []
for i in range(1000):
    my_env.reset()
    done = False
    steps = 0
    while not done:
#         my_env.render()
        obs, rew, done, _ = my_env.step(action=my_env.action_space.sample())
        steps += 1
    steps_array.append(steps)
my_env.close()

print('AVERAGE OF STEPS:', np.mean(steps_array))
print('STD OF STEPS:', np.round(np.std(steps_array), 2))

AVERAGE OF STEPS: 21.965
STD OF STEPS: 11.4


In [ ]:
video = io.open('./Gym Videos/' + game + '/' + openaigym.video.%s.video000000.mp4' % my_env.file_infix, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
    <video width="360" height="auto" alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
.format(encoded.decode('ascii')))

### Pole Velocity Error

This section pick an action based on the error function $E = p^2$ where $p$ is the pole velocity. I define the error in that way since we want a low pole velocity. Again, we do 1000 iterations an average the results.

In [7]:
#Define an error function using the pole velocity as the unique parameter to penalize the model
def error_function(obs):
    pos, vel, angle, pole_vel = obs
    error = (pole_vel**2)
    return error

In [9]:
#Exercise: Determine how many steps, on average, are taken, when
#actions are randomly sampled
steps_array = []
for i in range(1000):
    my_env.reset()
    action = my_env.action_space.sample()
    previous_error = -math.inf
    done = False
    steps = 0
    while not done:
#         my_env.render()
        obs, rew, done, _ = my_env.step(action=action)
        steps += 1
        new_error = error_function(obs)
        if new_error < previous_error:
            previous_error = new_error
            pass
        else:
            previous_error = new_error
            action = abs(action-1)
            
    steps_array.append(steps)
# my_env.close()
print('AVERAGE OF STEPS:', np.round(np.mean(steps_array), 2))
print('STD OF STEPS:', np.round(np.std(steps_array), 2))

AVERAGE OF STEPS: 157.64
STD OF STEPS: 21.66


Using this error function is clear that the number of steps is greater (much greater) that the randomness algorithm in average. Now, how can we improve the new record?